In [ ]:
import sys
DIR = "/home/tim/Research/torch-points3d/"
sys.path.insert(0, DIR)

from torch_points3d.datasets.grasp_classification import acronymvid
from torch_points3d.models.grasp_classification import minkowski
from torch_points3d.metrics.model_checkpoint import ModelCheckpoint

import matplotlib.pyplot as plt
import imageio

import os

In [ ]:
# ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-04/17-03-22"
# ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-04/18-14-55"
# ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-04/23-53-23"
# check_name="MinkUNet14A"


# ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-06/17-28-38"
# ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-12/10-51-01"
# check_name="STRes16UNet14B"

# ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-13/18-53-47"
# check_name="GraspSTRes16UNet14B"

# ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-16/14-23-59"
# check_name="GraspSTRes16UNet14B"

# from the overnight cluster run
# ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-21/01-06-13"
# check_name = "GraspMinkUNet14A"

ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-21/12-51-16"
check_name = "GraspMinkUNet14A"

In [ ]:
yaml_config = """
data:
    task: grasp_classification
    class: acronymvid.AcronymVidDataset
    name: acronymvid
    dataroot: /home/tim/Research/GraspRefinement/data
    process_workers: 8
    apply_rotation: False
    grid_size: 0.1
    mode: "last"

    train_pre_batch_collate_transform:
    - transform: ClampBatchSize
      params:
        num_points: 1000000

    train_transform:
    - transform: Random3AxisRotation
      params:
        apply_rotation: ${data.apply_rotation}
        rot_x: 8
        rot_y: 8
        rot_z: 180
    - transform: RandomSymmetry
      params:
        axis: [True, True, False]
    - transform: GridSampling3D
      params:
        size: ${data.grid_size}
        quantize_coords: True
        mode: ${data.mode}

    val_transform:
    - transform: GridSampling3D
      params:
        size: ${data.grid_size}
        quantize_coords: True
        mode: ${data.mode}
"""

In [ ]:
from omegaconf import OmegaConf
params = OmegaConf.create(yaml_config)

In [ ]:
dataset = acronymvid.AcronymVidDataset(params.data)

In [ ]:
import torch
device = torch.device("cpu")
# device = torch.device("cuda")

In [ ]:
model_checkpoint = ModelCheckpoint(ckpt_dir, check_name, "test", resume=False)

model = model_checkpoint.create_model(dataset, weight_name="latest").to(device)
model.eval()

dataset.create_dataloaders(model,
    batch_size=2,
    shuffle=False,
    num_workers=1,
    precompute_multi_scale=False
)

In [ ]:
loader = dataset.test_dataloaders[0]

In [ ]:
import pyvista as pv
import numpy as np
def plot_point_cloud(data, out_preds, out_coords, path="figs/out.gif"):
    p = pv.Plotter(notebook=True, shape=(1,2))
    p.open_movie(path)
    for t in np.unique(data.time.numpy()):
        p.clear()
        # plot ground truth on left
        p.subplot(0, 0)
        point_cloud = pv.PolyData(data.pos[data.time == t].numpy())
        point_cloud['y'] = data.y[data.time == t].numpy()
        p.add_points(point_cloud, show_scalar_bar=False, point_size=3)
        p.camera_position = [0, 3, -6]

        # plot prediction on right
        p.subplot(0, 1)
        out_pos = out_coords[:,-3:]
        out_time = out_coords[:,-4]
        point_cloud2 = pv.PolyData(out_pos[out_time == t])
        point_cloud2['y'] = np.array(out_preds[out_time == t] > 0, dtype=np.float)
        p.add_points(point_cloud2, show_scalar_bar=False, point_size=3)

        p.camera_position = [0, 3, -6]
        p.render()
        p.write_frame()
        # p.show(jupyter_backend='static')

In [ ]:
from tqdm import tqdm
with torch.no_grad():
    for i, data in enumerate(tqdm(loader)):
        # data = dataset.test_dataset[0][i]
        model.set_input(data, device)
        model.forward()

        # Extract first example from this batch
        example = model.data.get_example(0)
        # out_preds = model.output.features_at(0).detach().numpy()
        # out_coords = model.output.coordinates_at(0).detach().numpy()
        out_coords = model.input.coordinates.detach().cpu().numpy()

        out_preds = model.class_logits.detach().cpu().numpy()
        out_preds = out_preds[out_coords[:,0] == 0]
        out_preds = out_preds > 0

        out_coords = out_coords[out_coords[:,0] == 0]
        

        name = "figs/" + check_name + "".join(ckpt_dir.split('/')[-2:]) + f"/{i}.gif"
        os.makedirs(os.path.dirname(name), exist_ok=True)
        plot_point_cloud(example, out_preds, out_coords, path=name)

        
        # out = (out.reshape(-1, 300, 300) > 0).astype('uint8') * 255

        # for im in out:
        #     plt.imshow(im)
        #     plt.pause(0.01)

        # gt = data.y.detach().numpy().reshape(-1, 300, 300)
        # for im in gt:
        #     plt.imshow(im)
        #     plt.pause(0.01)

        # name = check_name + "".join(ckpt_dir.split('/')[-2:]) + f"_{i}.gif"
        # imageio.mimsave(f"figs/preds_{name}", out)

        # gtname = f"gt_{i}.gif"
        # imageio.mimsave(f"figs/{gtname}", data.y.detach().numpy().reshape(-1, 300, 300))

In [ ]:
torch.nn.functional.sigmoid(model.class_logits).max()

In [ ]:
import matplotlib.pyplot as plt

ax = plt.figure().add_subplot(projection='3d')

ad =  model.approach_dir.detach().cpu().numpy()
ax.quiver(0, 0, 0, ad[:,0], ad[:,1], ad[:,2], normalize=True)
ax.set_title("Approach Direction")

In [ ]:
n_bins = 20
fig, axs = plt.subplots(1, 3, sharey=True, tight_layout=True)

axs[0].hist(ad[:,0], bins=n_bins)
axs[1].hist(ad[:,1], bins=n_bins)
axs[2].hist(ad[:,2], bins=n_bins)

In [ ]:
import panel as pn
import pyvista as pv
from pyvista import examples
pn.extension('vtk', sizing_mode="stretch_width")
import numpy as np

# make point cloud from my data
data = example
t = 0
pos = data.pos[data.time == t].numpy()
point_cloud = pv.PolyData(pos, )
point_cloud['y'] = data.y[data.time == t].numpy()
point_cloud["radius"] = np.ones(len(pos))

# pyvista plotter
pl = pv.Plotter(notebook=True)
actor = pl.add_points(point_cloud, render_points_as_spheres=True, point_size=5)
pl.camera_position = [-1, 5, 10] #set camera position

# save initial camera properties
renderer = list(pl.ren_win.GetRenderers())[0]
initial_camera = renderer.GetActiveCamera()
initial_camera_pos = {"focalPoint": initial_camera.GetFocalPoint(),
                      "position": initial_camera.GetPosition(),
                      "viewUp": initial_camera.GetViewUp()}

# Panel creation using the VTK Scene created by the plotter pyvista
orientation_widget = True
enable_keybindings = True
vtkpan = pn.panel(pl.ren_win, sizing_mode='stretch_both', orientation_widget=orientation_widget,
                  enable_keybindings=enable_keybindings, height=600)
vtkpan

In [ ]:
from pyvista import demos

# basic glyphs demo
mesh = demos.glyphs(2)

text = demos.logo.text_3d("I'm interactive!", depth=0.2)
text.points *= 0.1
text.translate([0, 1.4, 1.5])
mesh += text
mesh['Example Scalars'] = mesh.points[:, 0]

mesh.plot(cpos='xy', jupyter_backend='ipygany', background='white',
          show_scalar_bar=True)